In [1]:
from biocrnpyler import *
from biocrnpyler.chemical_reaction_network import Species, Reaction, ChemicalReactionNetwork

Chemical reaction network for the violacein pathway.

$$\text{Tryptophan (TRP)} \leftrightarrow^{VioA} \text{IPA imine}$$
$$\text{2 IPA imine} \leftrightarrow^{VioB} \text{IPA imine dimer}$$
$$\text{IPA imine dimer} \leftrightarrow^{VioE} \text{Protodeoxyviolaceinic acid (PTDVA)}$$
$$\text{Protodeoxyviolaceinic acid (PTDVA)} \leftrightarrow^{VioD} \text{Protoviolaceinic acid (PTVA)}$$
$$\text{Protoviolaceinic acid (PTVA)} \leftrightarrow^{VioC} \text{Violacein (V)}\text{        (This is the desired end-product)}$$
$$\text{Protodeoxyviolaceinic acid (PTDVA)} \leftrightarrow^{VioC} \text{Deoxyviolacein (DV)}\text{        (This is a side-reaction)}$$





In [2]:
#DNA species
d_vioA = Species("vioA", material_type = "dna")
d_vioB = Species("vioB", material_type = "dna")
d_vioE = Species("vioE", material_type = "dna")
d_vioD = Species("vioD", material_type = "dna")
d_vioC = Species("vioC", material_type = "dna")

#Transcript species
t_vioA = Species("vioA", material_type = "transcript") 
t_vioB = Species("vioB", material_type = "transcript") 
t_vioE = Species("vioE", material_type = "transcript") 
t_vioD = Species("vioD", material_type = "transcript") 
t_vioC = Species("vioC", material_type = "transcript") 

#Enzyme species
VioA = Species("VioA", material_type = "enzyme")
VioB = Species("VioB", material_type = "enzyme")
VioE = Species("VioE", material_type = "enzyme")
VioD = Species("VioD", material_type = "enzyme")
VioC = Species("VioC", material_type = "enzyme")

#Pathway organic molecule species
TRP = Species("TRP", material_type = "organic_molecule")
IPA_imine = Species("IPA_imine", material_type = "organic_molecule")
IPA_imine_dimer = Species("IPA_imine_dimer", material_type = "organic_molecule")
PTDVA = Species("PTDVA", material_type = "organic_molecule")
PTVA = Species("PTVA", material_type = "organic_molecule")
DV = Species("DV", material_type = "organic_molecule")
V = Species("V", material_type = "organic_molecule")

#DNA assembly
G1_VioA = DNAassembly(str(d_vioA), promoter = "P", rbs = "RBS", transcript = t_vioA, protein = VioA)
G2_VioB = DNAassembly(str(d_vioB) , promoter = "P", rbs = "RBS", transcript = t_vioB, protein = VioB)
G3_VioE = DNAassembly(str(d_vioE) , promoter = "P", rbs = "RBS", transcript = t_vioE, protein = VioE)
G4_VioD = DNAassembly(str(d_vioD) , promoter = "P", rbs = "RBS", transcript = t_vioD, protein = VioD)
G5_VioC = DNAassembly(str(d_vioC) , promoter = "P", rbs = "RBS", transcript = t_vioC, protein = VioC)


VioB_kb = 10
VioB_ku = 1
VioB_kcat = 450
michalis_parameters = {("michalis_menten", "VioA", "kb"):10, ("michalis_menten", "VioA", "ku"):1, ("michalis_menten", "VioA", "kcat"):203,
                       ("michalis_menten", "VioB", "kb"):VioB_kb, ("michalis_menten", "VioB", "ku"):VioB_ku, ("michalis_menten", "VioB", "kcat"):VioB_kcat*VioB_kb/VioB_ku}    

mech_michalis = MichalisMenten()
specific_mechanisms = {mech_michalis.mechanism_type:mech_michalis}




#Enzyme assembly
E1_VioA = Enzyme(enzyme = VioA, substrate = "TRP", product = "IPA_imine", mechanisms = specific_mechanisms, parameters = michalis_parameters)
E2_VioB = MultiEnzyme(enzyme = VioB, substrates = ["IPA_imine", "IPA_imine"], products = ["IPA_imine_dimer"], mechanisms = specific_mechanisms, parameters = michalis_parameters)
E3_VioE = Enzyme(enzyme = VioE, substrate = "IPA_imine_dimer", product = "PTDVA")
E4_VioD = Enzyme(enzyme = VioD, substrate = "PTDVA", product = "PTVA")
E5_VioC = Enzyme(enzyme = VioC, substrate = "PTVA", product = "V")
E5_VioC_side = Enzyme(enzyme = VioC, substrate ="PTDVA", product = "DV")

#create a transcription and translation Mechanisms. 
mech_tx = SimpleTranscription()
mech_tl = SimpleTranslation()

#Enzyme mechanisms
mech_cat = BasicCatalysis()

#Mechanisms dictionary
default_mechanisms = {mech_tx.mechanism_type:mech_tx, mech_tl.mechanism_type:mech_tl, mech_cat.mechanism_type:mech_cat}

#Parameters
default_parameters = {"kb":100, "ku":10, "ktx":.5, "ktl":2, "kcat":100}


#Complete pathway
M_complete = TxTlExtract("e coli extract", components = [G1_VioA, G2_VioB, G3_VioE, G4_VioD, G5_VioC, E1_VioA, E2_VioB, E3_VioE, E4_VioD, E5_VioC, E5_VioC_side], parameters = default_parameters ,parameter_file = "default_parameters.txt")
CRN_complete = M_complete.compile_crn()
#print(repr(M_complete),"\n", CRN_complete.pretty_print(show_attributes = True, show_material = True, show_rates = True),"\n\n")


#liposome 1 TRP->PTVA pathway
M_liposome_1 = TxTlExtract("e coli extract", components = [G1_VioA, G2_VioB, G3_VioE, G4_VioD, E1_VioA, E2_VioB, E3_VioE, E4_VioD], parameters = default_parameters ,parameter_file = "default_parameters.txt")
CRN_liposome_1 = M_liposome_1.compile_crn()
sbml_liposome_1 = CRN_liposome_1.write_sbml_file('liposome_1.xml')


print(repr(M_liposome_1),"\n", CRN_liposome_1.pretty_print(show_attributes = True, show_material = True, show_rates = True),"\n\n")



#liposome 2 PTVA->V pathway + PTDVA -> DV side reaction
#M_liposome_2 = TxTlExtract("e coli extract", components = [G5_VioC, E5_VioC, E5_VioC_side], parameters = default_parameters ,parameter_file = "default_parameters.txt")
#CRN_liposome_2 = M_liposome_2.compile_crn()
#sbml_liposome_2 = CRN_liposome_2.write_sbml_file('liposome_2.xml')

M_liposome_2 = TxTlExtract("e coli extract", components = [G5_VioC, E5_VioC, E5_VioC_side], parameters = default_parameters ,parameter_file = "default_parameters.txt")
CRN_liposome_2 = M_liposome_2.compile_crn()
sbml_liposome_2 = CRN_liposome_2.write_sbml_file('liposome_2.xml')

print(repr(M_liposome_2),"\n", CRN_liposome_2.pretty_print(show_attributes = True, show_material = True, show_rates = True),"\n\n")




TxTlExtract: e coli extract
Components = [
	DNAassembly: dna_vioA
	DNAassembly: dna_vioB
	DNAassembly: dna_vioE
	DNAassembly: dna_vioD
	Enzyme: VioA
	MultiEnzyme: VioB
	Enzyme: VioE
	Enzyme: VioD
	Protein: RNAP
	Protein: Ribo
	Protein: RNAase ]
Mechanisms = {
	transcription:transcription_mm
	translation:translation_mm
	rna_degredation:rna_degredation_mm
	catalysis:michalis_menten
	binding:one_step_binding }
Global Mechanisms = { } 
 Species (36) = {0. complex[protein[RNAase]:transcript[vioA]], 1. transcript[vioA], 2. enzyme[VioA], 3. protein[RNAP], 4. complex[protein[Ribo]:transcript[vioA]], 5. protein[RNAase], 6. complex[dna[dna_vioA]:protein[RNAP]], 7. protein[Ribo], 8. dna[dna_vioA], 9. complex[protein[Ribo]:transcript[vioB]], 10. complex[protein[RNAase]:transcript[vioB]], 11. transcript[vioB], 12. enzyme[VioB], 13. complex[dna[dna_vioB]:protein[RNAP]], 14. dna[dna_vioB], 15. complex[protein[Ribo]:transcript[vioE]], 16. dna[dna_vioE], 17. transcript[vioE], 18. complex[dna[dna_vioE]:

In [3]:
# Creating an empty external subsystem

from subsbml import createBasicSubsystem, SimpleModel
import libsbml

external_ss, external_environment = createBasicSubsystem('external_environment', size = 1e-06 )

per_second = external_environment.createNewUnitDefinition(uId = 'per_second',uKind = libsbml.UNIT_KIND_SECOND,
                                                uExponent = -1)
substance = external_environment.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, uExponent = 1)


external_environment.createNewSpecies('PTDVA',sComp = 'external_environment', ListOfAmounts = 0, sConstant = False, sSubstance = 'substance')
external_environment.createNewSpecies('PTVA',sComp = 'external_environment', ListOfAmounts = 0, 
                             sConstant = False, sSubstance = 'substance')


# Write to SBML file 
_ = external_ss.writeSBML('external_environment.xml')


In [4]:
#Creating the aHL membrane for transporting PTVA
from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml


ahl_mb_ss = createNewSubsystem()
model1 = ahl_mb_ss.createNewModel('ahl_membrane','second','mole','substance')

ahl_mb = SimpleModel(model1)

per_second = ahl_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1) 
substance = ahl_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
ahl_mb.createNewCompartment('internal','internal',2e-06,'litre',True)
ahl_mb.createNewCompartment('external','external',1e-06,'litre',True)
##############################################################################################

ahl_mb.createNewSpecies( 'PTVA','internal',0,False,'substance')
ahl_mb.createNewSpecies( 'PTVA','external',0,False,'substance')
ahl_mb.createNewSpecies( 'PTDVA','internal',0,False,'substance')
ahl_mb.createNewSpecies( 'PTDVA','external',0,False,'substance')

ahl_mb.createNewParameter( 'kb',10,False,'per_second')
ahl_mb.createNewParameter( 'kd',2,False,'per_second')

model1 = ahl_mb_ss.getSBMLDocument().getModel()

#PTVA(internal) <--> PTVA(external)
r1_sbml = model1.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1')
r1.setReversible(False)
r1.createNewReactant('PTVA',False,1)
r1.createNewProduct('PTVA_1', False, 1)
r1.createRate('kb * PTVA - kd * PTVA_1')


#PTDVA(internal)<--> PTDVA(external)
r2_sbml = model1.createReaction()
r2 = SimpleReaction(r2_sbml)
r2.setId('r2')
r2.setReversible(True)
r2.createNewReactant('PTDVA',False,1)
r2.createNewProduct('PTDVA_1', False, 1)
r2.createRate('kb * PTDVA - kd * PTDVA_1')

# Write to SBML file 
_ = ahl_mb_ss.writeSBML('membrane_ahl_simple1.xml')
print('Model written to a SBML file successfully')






Model written to a SBML file successfully


In [5]:
#Creating the aHL membrane for transporting PTDVA and PTVA

ahl_mb_ss2 = createNewSubsystem()
model2 = ahl_mb_ss2.createNewModel('ahl_membrane','second','mole','substance')

ahl_mb2 = SimpleModel(model2)

per_second = ahl_mb2.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1) 
substance = ahl_mb2.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
ahl_mb2.createNewCompartment('internal','internal',2e-06,'litre',True)
ahl_mb2.createNewCompartment('external','external',1e-06,'litre',True)
##############################################################################################

ahl_mb2.createNewSpecies( 'PTVA','external',0,False,'substance')
ahl_mb2.createNewSpecies( 'PTVA','internal',0,False,'substance')
ahl_mb2.createNewSpecies( 'PTDVA','external',0,False,'substance')
ahl_mb2.createNewSpecies( 'PTDVA','internal',0,False,'substance')

ahl_mb2.createNewParameter( 'kb',2,False,'per_second')
ahl_mb2.createNewParameter( 'kd',10,False,'per_second')

ahl_mb_ss2.getSBMLDocument().getModel()

#PTVA(external) <--> PTVA(internal)
r3_sbml = model2.createReaction()
r3 = SimpleReaction(r3_sbml)
r3.setId('r3')
r3.setReversible(True)
r3.createNewReactant('PTVA',False,1)
r3.createNewProduct('PTVA_1', False, 1)
r3.createRate('kb * PTVA - kd * PTVA_1')


#PTDVA(internal)<--> PTDVA(external)
r4_sbml = model2.createReaction()
r4 = SimpleReaction(r4_sbml)
r4.setId('r4')
r4.setReversible(True)
r4.createNewReactant('PTDVA',False,1)
r4.createNewProduct('PTDVA_1', False, 1)
r4.createRate('kb * PTDVA - kd * PTDVA_1')

# Write to SBML file 
_ = ahl_mb_ss2.writeSBML('membrane_ahl_simple2.xml')
print('Model written to a SBML file successfully')







Model written to a SBML file successfully
